In [1]:
##update the metabolite ids and the reaction ids to contain only _c and not _c0
import cobra
from cobra import Metabolite, Reaction, Gene

In [2]:
#load the most recent version of the model
pst = cobra.io.read_sbml_model("../results/v4_with_all_annotations.xml")

In [11]:
#remove all exterior reactions, aka the exchange and supp biomass equations
#without removal, metabolites from these will interfere with code later, due to updating the metabolite list, errors include "metabolite already exists" but does not pass it
pst.reactions.PAO1_Biomass.remove_from_model(remove_orphans = True)
pst.reactions.PA_Biomass_v13.remove_from_model(remove_orphans = True)
pst.reactions.PA_Biomass_v13ub.remove_from_model(remove_orphans = True)
pst.reactions.PA_Biomass_v4.remove_from_model(remove_orphans = True)
for reaction in pst.reactions:
    if "LPAREN" in reaction.id:
        reaction.remove_from_model(remove_orphans = True)
        print ('yes')
    else:
        pass
    if "PJT" in reaction.id:
        reaction.remove_from_model(remove_orphans = True)
    else:
        pass
# for some reason the above code does not iterate over all of the values at the same time, so must be exicuted multiple times to get ride of all the expern reactions

In [12]:
#manual check for the reactions and metabolites that may still be present
#if the previous chunk is exicuted multiple times, to where only 1 "yes" appears twice in a row, both of these should print nothing
for reaction in pst.reactions:
    if "LPAREN" in reaction.id:
        print (reaction)
for reaction in pst.reactions:
    if "PJT" in reaction.id:
        #print(reaction)
        print (reaction.id)

In [13]:
#call all of the metabolite ids and reduce the names to only _c or _e
#this is done to standardize the metabolite names, as previously there was a mix of _c and _c0
#the repeated_ids list should render blank
repeated_ids = []

for met in pst.metabolites:
    metid= met.id
    if '_c0' in metid:
        metid = metid[:-1]
        if metid not in pst.metabolites:
            met.id = metid
        else:
            repeated_ids.append(metid)
    if '_e0' in metid:
        metid = metid[:-1]
        if metid not in pst.metabolites:
            met.id = metid
        else:
            repeated_ids.append(metid)
#repeated ids are cooming from interferences with the biomass functions
print (repeated_ids)
    

[]


In [14]:
#add back the necessary exchange reactions that were previously deleted
#each exchange reaction is based on whetehr a _e is in the metaboltie descriptor
for met in pst.metabolites:
    if "_e" in met.id:
        reaction = Reaction('EX_' + met.id)
        reaction.name = 'Exchange' + met.id[:-3]
        reaction.subsystem = 'Exchange'
        reaction.lower_bound = 0.
        reaction.upper_bound = 1000.
        reaction.add_metabolites({met:-1})
        pst.add_reactions([reaction])
    else:
        pass

cpd00794_e
EX_cpd00794_e: cpd00794_e --> 
cpd00023_e
EX_cpd00023_e: cpd00023_e --> 
cpd00034_e
EX_cpd00034_e: cpd00034_e --> 
cpd00041_e
EX_cpd00041_e: cpd00041_e --> 
cpd00048_e
EX_cpd00048_e: cpd00048_e --> 
cpd00053_e
EX_cpd00053_e: cpd00053_e --> 
cpd00080_e
EX_cpd00080_e: cpd00080_e --> 
cpd00107_e
EX_cpd00107_e: cpd00107_e --> 
cpd00118_e
EX_cpd00118_e: cpd00118_e --> 
cpd00119_e
EX_cpd00119_e: cpd00119_e --> 
cpd00154_e
EX_cpd00154_e: cpd00154_e --> 
cpd00156_e
EX_cpd00156_e: cpd00156_e --> 
cpd00210_e
EX_cpd00210_e: cpd00210_e --> 
cpd00264_e
EX_cpd00264_e: cpd00264_e --> 
cpd00322_e
EX_cpd00322_e: cpd00322_e --> 
cpd00637_e
EX_cpd00637_e: cpd00637_e --> 
cpd00205_e
EX_cpd00205_e: cpd00205_e --> 
cpd00971_e
EX_cpd00971_e: cpd00971_e --> 
cpd00067_e
EX_cpd00067_e: cpd00067_e --> 
cpd00060_e
EX_cpd00060_e: cpd00060_e --> 
cpd00129_e
EX_cpd00129_e: cpd00129_e --> 
cpd00027_e
EX_cpd00027_e: cpd00027_e --> 
cpd00054_e
EX_cpd00054_e: cpd00054_e --> 
cpd00009_e
EX_cpd00009_e: cpd00009

In [15]:
#add back the appropriate biomass function with its metabolites, this will be biomass v4, eventually
#I will hold off on adding this for now due to the presence of manually added metabolites that do not have the proper seed id
cobra.io.write_sbml_model(pst, "../results/suffix_fixed_no_biomass.xml")